In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
%load_ext memory_profiler

ModuleNotFoundError: No module named 'memory_profiler'

In [2]:
import numpy as np
import pandas as pd
import scanpy.api as sc

/home/ahsvargo/miniconda3/envs/r35py37/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [3]:
import anndata

In [4]:
import sys

In [5]:
from scipy.stats import rankdata
import scipy.sparse as spsp

## Load the vectors of constants previously calculated

In [6]:
consts = [np.load("1M-fold{}-rankConsts.npz".format(i))['consts'] for i in range(5)]
consts = np.array(consts)

In [7]:
consts.shape

(5, 39, 24015)

In [8]:
adata = sc.read_h5ad("scanpy/fold0/1M-fold0.h5ad")
sc.logging.print_memory_usage()

Memory usage: current 16.14 GB, difference +16.14 GB


Not sure why I need the full fold data... I need to get the y vector, I think.  Thus, load the data in `backed` mode

In [9]:
marks = np.load("scanpy/fold0/1M-fold0-method.npz")['marks']

In [8]:
adata = sc.read_h5ad("1M-nzGenes-clusts.h5ad", backed='r')
sc.logging.print_memory_usage()

Memory usage: current 0.35 GB, difference +0.35 GB


In [9]:
adata

AnnData object with n_obs × n_vars = 1306127 × 24015 backed at '1M-nzGenes-clusts.h5ad'
    obs: 'louvain', 'graphclust'
    var: 'gene_ids', 'n_counts'

In [9]:
yVec = np.array([int(clust) for clust in adata.obs['louvain'].values])

In [10]:
folds = np.load("10x-5folds.npz")
folds = [folds["fold{}".format(i)] for i in range(5)]

In [11]:
clusterindices = {}
for k in range(yVec.max()):
    clusterindices[k] = np.where(yVec == k)[0]

In [12]:
def tauStd(clust, fold, yVec=yVec, clusterindices=clusterindices, highval=1, lowval=-1):
    
    foldSize = fold.shape[0]
    foldY = yVec[fold]
    foldClust = np.where(foldY == clust)[0]
    
    tau = lowval*np.ones(foldSize)
    tau[foldClust] = highval
    
    return np.std(tau)    

In [68]:
list(map(lambda x: tauStd(x, folds[0]), range(yVec.max())))

[0.573345720874866,
 0.5685789761157408,
 0.5057922397354279,
 0.4876263066065458,
 0.48403123173197427,
 0.43168685639917587,
 0.4296170369884458,
 0.4201727657003719,
 0.4027045481701002,
 0.3855117012890736,
 0.37925853754905925,
 0.3734345936979563,
 0.3707800098838256,
 0.35436812561090764,
 0.35136350517682957,
 0.32745243989371975,
 0.32877497093855673,
 0.29796840956774084,
 0.2824942716295407,
 0.27223096670161995,
 0.2613055440526384,
 0.235197240603585,
 0.232074683015444,
 0.22949234239899552,
 0.22027021583886833,
 0.18280768869911504,
 0.1806534387334112,
 0.16347156299899912,
 0.16337912600944238,
 0.13768778887315455,
 0.06841132365654295,
 0.06021419296159305,
 0.0505523286631776,
 0.047430557888153126,
 0.04528587640845992,
 0.039512733569508136,
 0.041032377346178074,
 0.03873052235929032]

Did this work?  The below manual calculations make it appear that it did.

In [64]:
clust = 2
foldN = 0
clustSize = np.where(yVec[folds[foldN]] == clust)[0].shape[0]

mu = (2*clustSize - folds[foldN].shape[0])/folds[foldN].shape[0]
lowval = -1 - mu
highval = 1 - mu

np.sqrt((clustSize*highval**2 - clustSize*lowval**2 + folds[foldN].shape[0]*lowval**2)/folds[foldN].shape[0])

0.5057922397354279

In [56]:
(highval*clustSize + lowval*(folds[foldN].shape[0]-clustSize))/folds[foldN].shape[0]

0.0

We want to compare a loop to a map.  The map requires us to allocate a vector every time; the loop does not.

In [13]:
# soft threshold inVec by decreasing all entries by param
def softThreshold(inVec, param):
    
    signs = np.sign(inVec)
    inVec = inVec - param * signs
    inVec[ np.invert(signs == np.sign(inVec)) ] = 0
    return inVec

In [14]:
# input the consts and the index to get the 1-norm of the unit version of the soft-thesholded consts vector
def stsScore(consts, ind):
    
    stVec = softThreshold(consts, np.abs(consts[ind]))
    norm = np.linalg.norm(stVec)
    norm = norm if norm > 0 else 1
    return np.abs( stVec / norm ).sum()

# input the soft-thresholded vector to get the 1-norm of the unit version of the input
def sScore(stVec):
    norm = np.linalg.norm(stVec)
    norm = norm if norm > 0 else 1
    return np.abs( stVec / norm ).sum()

In [56]:
def sScores(consts):

    return np.array(list(map( lambda ind: stsScore(consts, ind), range(consts.shape[0]) )))

In [58]:
%%time
fold0 = np.array(list(map( sScores, consts[0] )))

CPU times: user 1h 3min 39s, sys: 7h 13min 48s, total: 8h 17min 27s
Wall time: 1h 2min 45s


In [67]:
np.savez("rc-svals-map4.npz", svals=fold4)

In [66]:
%%time
fold4 = np.array(list(map( sScores, consts[4] )))

CPU times: user 1h 3min 20s, sys: 7h 15min 21s, total: 8h 18min 41s
Wall time: 1h 2min 53s


In [68]:
fullrc = np.array([fold0, fold1, fold2, fold3, fold4])
fullrc.shape

(5, 39, 24015)

In [69]:
np.savez("rc-svals-map.npz", svals=fullrc)

Having a test for $\| \mathrm{vec} \|_2 = 0$ doen't lose time and is useful later on:

Edit... nvm with more cores for some reason?  Actually no idea.  There is huge variance in the time that these commands take.  Like $\pm$ 15 seconds.  I think that there might be large communication issues.  The largest time difference between runs seems to occur in `numpy.core.multiarray.dot`.  I'm assuming that comes from the call to `np.linalg.norm`.  In any case, I know that `dot` is parallelized.  So that must be the issue

In [18]:
%%prun
normVals = np.array(list(map( lambda ind: stsScore(consts[0,1], ind), range(consts[0,1].shape[0]) )))

In [15]:
%%time
unnormVals = np.array(list(map( lambda ind: stsScore(consts[0,1], ind), range(consts[0,1].shape[0]) )))
#unnormVals[np.isnan(unnormVals)] = 0


CPU times: user 1min 27s, sys: 10min 8s, total: 11min 35s
Wall time: 1min 27s


In [106]:
unnormVals[np.isnan(unnormVals)] = 0

In [55]:
np.any(np.isnan(unnormVals))

False

In [19]:
np.nonzero(np.array(normVals) - np.array(unnormVals))

(array([], dtype=int64),)

In [20]:
np.min(unnormVals)

0.0

In [98]:
normVals[4814]

0.0

In [100]:
unnormVals[4814]

numpy.float64

Here is the `for` loop version:

In [124]:
%%prun
args = np.argsort(np.abs(consts[0,0]))

stVec = consts[0,0].copy()
prevParam = 0
outputs = np.zeros(stVec.shape[0])

for arg in args:
    stVec = softThreshold(stVec, np.abs(consts[0,0,arg]) - prevParam)
    prevParam = np.abs(consts[0,0,arg])
    outputs[arg] = sScore(stVec)
    

In [23]:
def sScoresClust(consts):
    args = np.argsort(np.abs(consts))

    stVec = consts.copy()
    prevParam = 0
    outputs = np.zeros(stVec.shape[0])

    for arg in args:
        stVec = softThreshold(stVec, np.abs(consts[arg]) - prevParam)
        prevParam = np.abs(consts[arg])
        outputs[arg] = sScore(stVec)
        
    return outputs

In [24]:
%%time
outputsTest = sScoresClust(consts[0,1])

CPU times: user 1min 20s, sys: 8min 58s, total: 10min 18s
Wall time: 1min 17s


In [44]:
prevParam = np.abs(consts[0,1][args[-2]])
stVec = softThreshold(consts[0,1], np.abs(consts[0,1][args[-1]]) - prevParam)
sScore(stVec)

65.56289096495613

In [46]:
consts[0,1][args[0]]

0.0

In [29]:
args = np.argsort(np.abs(consts[0,1]))

In [26]:
outputsTest[0]

42.03042629700728

In [27]:
unnormVals[0]

42.03042629700738

In [28]:
np.min(outputsTest)

1.0

In [32]:
np.nonzero(outputsTest - unnormVals)[0].shape

(22061,)

In [37]:
firstInd = -1
currentInd = 0
firstRun = 0
for i,arg in enumerate(args):
    if outputsTest[arg] != unnormVals[arg]:
        if firstInd < 0:
            firstInd = i
            currentInd = i
            firstRun = 1
        elif i != currentInd + 1:
            print("Doesn't happen in a chunk")
            break
        else:
            currentInd = i
            firstRun += 1

print("{} {}".format(firstInd,firstRun))

Doesn't happen in a chunk
165 1


In [52]:
outputsTest[args[-4]]- unnormVals[args[-4]]

5.3734794391857577e-14

In [130]:
np.nonzero(outputsTest - outputs)

(array([], dtype=int64),)

In [29]:
%%time
fold4 = list(map(  lambda clust: sScoresClust(consts[4, clust]), range(consts.shape[1])  ))

CPU times: user 54min 36s, sys: 6h 11min 54s, total: 7h 6min 31s
Wall time: 43min 13s


In [20]:
fold0 = np.array(fold0)

In [24]:
fold1 = np.array(fold1)

In [26]:
fold2 = np.array(fold2)

In [28]:
fold3 = np.array(fold3)

In [30]:
fold4 = np.array(fold4)

In [60]:
np.savez("10x-rankCorr-sValues.npz", svals=np.array([fold0, fold1, fold2, fold3, fold4]))

In [61]:
stuff = np.load("10x-ranks-sValues.npz")['svals']

In [62]:
rstuff = np.load("10x-rankCorr-sValues.npz")['svals']

In [159]:
np.nonzero(stuff[0] - fold0)

(array([], dtype=int64), array([], dtype=int64))

Any difference between ranks and rankCorr?

In [64]:
stuff.shape

(5, 39, 24015)

In [75]:
nMarks = 10000
for ind in range(5):
    print( len(list( set(np.argsort(stuff[0,38])[:nMarks]) & set(np.argsort(rstuff[0,38])[:nMarks]) )) )

10000
10000
10000
10000
10000


Dey da same.  Dat not so good.  

## Exploration and bug testing

This was carried out on the `ranks` data.  The smallest `s` values ($\ell_1$ norm of softThresholded vector) should correspond to the largest entries of the original constants.  This is simply a processed version of those constants.

In [149]:
[np.argsort(goo[0])[:10] for goo in [fold0, fold1, fold2, fold3, fold4]]

[array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563])]

In [153]:
[np.flipud(np.argsort(np.abs(consts[i,0])))[:10] for i in range(5)]

[array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563])]

Here is on `rankCorr` - looks the same.

In [32]:
[np.argsort(goo[0])[:10] for goo in [fold0, fold1, fold2, fold3, fold4]]

[array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563])]

In [33]:
[np.flipud(np.argsort(np.abs(consts[i,0])))[:10] for i in range(5)]

[array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563]),
 array([ 4814,   331,  3778, 15116, 17810, 13564, 14349,  5811, 21437,
         7563])]

Average difference between first two folds on cluster 0: pretty significant.

In [34]:
np.abs(consts[0,0] - consts[1,0]).mean()

62582834.24989936

How close are the $s$ values that we calculate for the same gene in the same cluster across all of the folds?  Looks pretty good to be honest.

In [78]:
[goo[30,args[10000]] for goo in [fold0, fold1, fold2, fold3, fold4]]

[50.45545124829106,
 47.658500198626896,
 47.63315681519226,
 35.46718824343834,
 47.70128033711569]

In [79]:
[goo[30,args[10000]] for goo in stuff]

[50.45545124829116,
 47.65850019862692,
 47.63315681519222,
 35.46718824343819,
 47.70128033711571]

In [36]:
[np.argsort(goo[0])[-1] for goo in [fold0, fold1, fold2, fold3, fold4]]

[1604, 18718, 3091, 16839, 8094]

In [51]:
consts[0,0,422]

808582.9591355842

Examine some of the constants.  The smallest constants are negative, we must sort the abs.

In [38]:
args = np.argsort(np.abs(consts[0,0]))

In [53]:
np.where(args[:70] == 1604)

(array([63]),)

In [54]:
args[:70]

array([ 1044, 15791, 11572,  3680,  9554,  3233,  6948, 12430, 11330,
       18601,  4922, 10842,  6504, 14426,  6069, 23696, 20714, 23694,
        4197, 10838, 22256, 17349, 15979,   795,  2524, 14482, 14622,
       10408, 19010,  5208, 10413,  1248, 15515,  3263, 19373,  9637,
        3084, 12371,   119,  8858,  4137,  4138, 10123,  9254, 13253,
        7995,  4736, 15488,  9616, 16348,  4338,  2303,  8293, 18518,
        8196, 18517, 18516,  5050, 17870,  8696,  3381,  5011, 14402,
        1604, 19227, 14388,  7841,  7276,  7843,  7855])

In [55]:
np.abs(consts[0,0]/np.linalg.norm(consts[0,0])).sum()

86.44149863003287

In [59]:
fold0[0,1604]

86.44149863003287

In [36]:
consts[0,0][args[161]]

-0.009432268546976926

In [50]:
args[20]

22256

In [61]:
args[2000]

15331

Developing the looped method and comparing to the mapped method to make sure that we get the same results.  `outputs` contains the looped results for the iterative soft thresholding.

In [68]:
prevParam = 0
total = 0
for ind in range(2000):
    total += np.abs(consts[0,0,args[ind]]) - prevParam
    prevParam = np.abs(consts[0,0,args[ind]])
    

In [74]:
stVec = softThreshold(consts[0,0], np.abs(consts[0,0, args[2000]]))

In [78]:
sScore(stVec) - outputs[args[2000]]

0.0

Clearly some sort of issue here.  I think that we may have solved it further along.  Keep reading.

In [79]:
stsScore(consts[0,0], args[2000])

88.18845360103252

In [77]:
outputs[args[2000]]

84.722641528138

The `prevParam` method is getting the correct total amount of softThresholding.

In [72]:
total - np.abs(consts[0,0,args[2000]])

0.0

Careful - this loads all of the data in to memory.  The `backed` data cannot be sliced without loading it all it seems.

In [39]:
testcol = adata.X[:,args[160]]

In [40]:
testcol.data

array([], dtype=float32)

We now are matching with the normVals amounts.  So life is good. (`normVals` is created by a map with `stsScores` above).

In [83]:
outputs[args[2000]] - normVals[args[2000]]

0.0

In [85]:
(outputs - np.array(normVals)).max()

2.913225216616411e-13

In [22]:
(outputsTest - np.array(normVals)).max()

2.1316282072803006e-13

How many markers we would select for cluster 0 with a cutoff of 400.

In [122]:
np.where(normVals < 20)[0].shape[0]

927